In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
root_path = '/content/drive/My Drive/Colab Notebooks/'  #change dir to your project folder
#!ls '/content/drive/My Drive/Colab Notebooks/'

In [ ]:
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import cv2
import numpy as np
from PIL import Image

Using TensorFlow backend.


In [ ]:
#variables
num_classes = 7 #angry, disgust, fear, happy, sad, surprise, neutral
batch_size = 32
epochs = 64

dropout=0.5
fc_layers = [512,512]

img_height = 48
img_width = 48

In [ ]:
#construct CNN structure
base_model = ResNet50(weights= 'imagenet', include_top=False, input_shape= (img_height,img_width,3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
def build_model(base_model, dropout, fc_layers, num_classes):
    # Taking the output of the last convolution block in ResNet50
    x = base_model.output

    # Flatten the output
    x = Flatten()(x)

    # Adding a fully connected layer
    for fc in fc_layers:
        x = Dense(fc, activation='relu')(x) 
        x = Dropout(dropout)(x)
    
    # Adding a fully connected layer having 7 neurons which will
    # give the probability of image having angry, disgust, fear, happy, sad, surprise or neutral
    predictions = Dense(num_classes, activation='softmax')(x) 
    
    # Model to be trained
    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [ ]:
TRAIN_DIR = root_path + 'FER/FER2013/Training/'
VAL_DIR = root_path + 'FER/FER2013/PrivateTest/'
TEST_DIR = root_path + 'FER/FER2013/PublicTest/'

train_datagen =  ImageDataGenerator()
val_datagen =  ImageDataGenerator()
test_datagen =  ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, 
    target_size=(img_height, img_width), 
    batch_size=batch_size, 
    class_mode='categorical')

val_generator = train_datagen.flow_from_directory(
    VAL_DIR, 
    target_size=(img_height, img_width), 
    batch_size=batch_size, 
    class_mode='categorical')

test_generator = train_datagen.flow_from_directory(
    TEST_DIR, 
    target_size=(img_height, img_width), 
    batch_size=batch_size, 
    class_mode='categorical')

Found 28735 images belonging to 7 classes.
Found 3607 images belonging to 7 classes.
Found 3608 images belonging to 7 classes.


In [ ]:
model = build_model(base_model, dropout=dropout, fc_layers=fc_layers, num_classes=num_classes)

In [ ]:
adam = Adam(lr=0.0001)
model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 48, 48, 3)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 54, 54, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 24, 24, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 24, 24, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [ ]:
history = model.fit_generator(train_generator, epochs=epochs, workers=8, steps_per_epoch=100, shuffle=True)
model.save(root_path + 'saved/my_model18.h5')

Epoch 1/64
100/100 [==============================] - 227s 2s/step - loss: 2.5326 - accuracy: 0.1878
Epoch 2/64
100/100 [==============================] - 206s 2s/step - loss: 1.9963 - accuracy: 0.2382
Epoch 3/64
100/100 [==============================] - 203s 2s/step - loss: 1.8519 - accuracy: 0.2719
Epoch 4/64
100/100 [==============================] - 206s 2s/step - loss: 1.7604 - accuracy: 0.3025
Epoch 5/64
100/100 [==============================] - 205s 2s/step - loss: 1.6600 - accuracy: 0.3547
Epoch 6/64
100/100 [==============================] - 203s 2s/step - loss: 1.6062 - accuracy: 0.3741
Epoch 7/64
100/100 [==============================] - 204s 2s/step - loss: 1.5767 - accuracy: 0.3900
Epoch 8/64
100/100 [==============================] - 203s 2s/step - loss: 1.5453 - accuracy: 0.4097
Epoch 9/64
100/100 [==============================] - 204s 2s/step - loss: 1.5395 - accuracy: 0.4006
Epoch 10/64
100/100 [==============================] - 199s 2s/step - loss: 1.4329 - accura

In [ ]:
val_loss, val_acc = model.evaluate_generator(val_generator, steps=1, verbose=0)
print(val_loss)
print(val_acc)

1.5440828800201416
0.65625


In [ ]:
loss, acc = model.evaluate_generator(test_generator, steps=1, verbose=0)
print(loss)
print(acc)

0.8289023637771606
0.78125
